In [0]:
import io
import re as re
import zipfile as zipfile
import math
import random

In [306]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
commentsW = {}      # { (doc, emot): comment, ... }
commentCounter = {}

mytextzip = ''
path = '/content/drive/My Drive/film_yorumlari.zip'

emotions = {
    "1": "pozitif",
    "2": "tarafsiz",
    "3": "negatif"
}

with zipfile.ZipFile(path) as z:
    for zipinfo in z.infolist():
        if zipinfo.filename.endswith('.txt') and re.search('raw_texts', zipinfo.filename):
            with z.open(zipinfo) as f:
                textfile = io.TextIOWrapper(f, encoding='cp1254', newline='')
                for line in textfile:
                    if len(line.strip()): mytextzip += line.strip()

                filename = str(zipinfo.filename)
                parts = filename.split("/")
                hash = parts[-2]
                for key, emot in emotions.items():
                    if hash == emot:
                        hash = key
                        break
                parts = parts[-1].split(".")
                doc = parts[0]
                
                commentsW[(doc, hash)] = mytextzip
                if hash not in commentCounter:
                    commentCounter[hash] = 0
                commentCounter[hash] += 1
                mytextzip = ''

In [0]:
def splitTrainTest(dict, n):
    train = {}      # { emot: [text1's (doc, emot) value, text2's (doc, emot) value,...], ... }
    test_e = []     # [ 1. text's emot, ... ]
    test_t = []     # [ 1. text's (doc, emot) value, ... ]

    temp = {}       # to keep random index to selecting which texts will be test text
    for emot, number in commentCounter.items():
        temp[emot] = []
        while len(temp[emot]) < n:
            x = random.randint(0, number-1)
            if x not in temp[emot]:
                temp[emot].append(x)
        temp[emot] = sorted(temp[emot])

    # temp = {'3': [21, 33], '1': [13, 25], '2': [2, 25]}

    for doc, emot in dict.keys():
        if emot not in train:
            train[emot] = []

        if len(train[emot]) in temp[emot]:
            temp[emot].remove(len(train[emot]))
            for j, num in enumerate(temp[emot]):
                temp[emot][j] -= 1
            test_t.append((doc, emot))
            test_e.append(emot)   # to prediction
        else:
            train[emot].append((doc, emot))

    return train, test_e, test_t

In [0]:
train, test_e, test_t = splitTrainTest(commentsW, 4)

In [310]:
print(train)
print(test_e)
print(test_t[0])

{'3': [('1', '3'), ('10', '3'), ('11', '3'), ('12', '3'), ('14', '3'), ('16', '3'), ('17', '3'), ('18', '3'), ('19', '3'), ('2', '3'), ('20', '3'), ('21', '3'), ('22', '3'), ('24', '3'), ('25', '3'), ('26', '3'), ('27', '3'), ('28', '3'), ('29', '3'), ('3', '3'), ('30', '3'), ('31', '3'), ('32', '3'), ('33', '3'), ('34', '3'), ('35', '3'), ('4', '3'), ('6', '3'), ('7', '3'), ('8', '3'), ('9', '3')], '1': [('1', '1'), ('10', '1'), ('11', '1'), ('12', '1'), ('13', '1'), ('14', '1'), ('15', '1'), ('16', '1'), ('18', '1'), ('19', '1'), ('2', '1'), ('20', '1'), ('21', '1'), ('22', '1'), ('23', '1'), ('24', '1'), ('25', '1'), ('26', '1'), ('27', '1'), ('28', '1'), ('29', '1'), ('3', '1'), ('30', '1'), ('32', '1'), ('34', '1'), ('35', '1'), ('4', '1'), ('5', '1'), ('7', '1'), ('8', '1'), ('9', '1')], '2': [('10', '2'), ('11', '2'), ('12', '2'), ('13', '2'), ('14', '2'), ('15', '2'), ('16', '2'), ('17', '2'), ('18', '2'), ('19', '2'), ('2', '2'), ('20', '2'), ('21', '2'), ('22', '2'), ('24', '

In [0]:
def splitIntoWords(text):
    text = text.strip()

    text = text.replace("(","")
    text = text.replace(")","")

    text = text.replace("”",'"')
    text = text.replace(',"','"')
    text = text.replace(",", "")
    text = text.replace("\n", ".")
    text = text.replace("  ", " ")

    text = re.split(r'\W', text.lower())

    return text

In [0]:
def createDB(emot, docs, DB, n):

    temp = []
    for doc, emot in docs:
        text = commentsW[(doc, emot)]
        temp.append(splitIntoWords(text))

    # DB = { emot1: { word1: [ 'word1's count in emot1 texts , {word2: 'word1 word2' pairs' count in emot1 texts, ...} ], ... }, ...}
    # DB = { "3": { 'senaryo': [3, {'yazılmaz': 1, 'oyunculuk': 1, 'sorunu': 1}], ... }

    DB[emot] = {}

    for text in temp:
        for i, word in enumerate(text):
            if i == len(text)-n:
                break
            else:
                for j in range(n-1):
                    if j == 0:
                        continue
                    word += " " + text[i+j]

                if word not in DB[emot]:
                    DB[emot][word] = []
                    DB[emot][word].append(0)
                    DB[emot][word].append({})
                
                after = text[i+n-1]           # next word
                DB[emot][word][0] += 1      # word1's count in text

                # DB[emot][word][1]  dictionary contains words that come after word1 as key and their counts as value

                if after not in DB[emot][word][1]:
                    DB[emot][word][1][after] = 1
                else:
                    DB[emot][word][1][after] += 1

    return DB

In [313]:
# Create DB

n = int(input('Enter n: '))
DB = {}
for emot, docs in train.items():
    DB = createDB(emot, docs, DB, n)

# n = 2
# print(DB["3"])
# {'senariste': [1, {'lafım': 1}], 'lafım': [2, {'rain': 1, 'filmin': 1}], 'rain': [2, {'man': 2}], ...}


# n = 3
# print(DB["3"])
# {'senariste lafım': [1, {'rain': 1}], 'lafım rain': [1, {'man': 1}], 'rain man': [2, {'filmini': 1,...}], ...}

Enter n: 2


In [0]:
predictions = []

for doc, emot in test_t:
    pr = {}     # to find best emotion prediction for text
                # { emotion: prediction value, ... }  
    
    txt = commentsW[(doc, emot)]
    words = splitIntoWords(txt)

    for emot in train.keys():

        pr[emot] = 0

        for i, word in enumerate(words):
            if i == len(words)-n:
                break

            for j in range(n-1):
                if j == 0:
                    continue
                word += " " + words[i+j]

            after = words[i+n-1]        # next word

            if word in DB[emot]:
                cTotal = DB[emot][word][0]             # of "word" in train text
                if after in DB[emot][word][1]:
                    cPart = DB[emot][word][1][after]   # of "word after" in train text

                    p = cPart / cTotal      # prediction
                else:
                    p = 1e-10
            else:
                p = 1e-10

            # to prevent errors;
            # if one of words(word or after) not in emotion's texts,
            # p will equal very small number

            pr[emot] += math.log(p)
            
    # to find the best emotion prediction
    pr = sorted(pr.items(), key=lambda pr: pr[1], reverse=True)
    # pr[0][0] --> the best emotion
    predictions.append(pr[0][0])

In [315]:
print(predictions)

['3', '2', '3', '3', '1', '2', '1', '1', '1', '3', '2', '3']


In [316]:
acc = 0
for i in range(len(predictions)):
    if predictions[i] == test_e[i]:
        acc += 1

print(acc, "/", len(predictions))
print("accuracy:", (acc / len(predictions)))

# n = 2
# 9 / 15    ---> accuracy: 0.6
# 6 / 12    ---> accuracy: 0.5
# 7 / 12    ---> accuracy: 0.58

# n = 3
# 5 / 15    ---> accuracy: 0.33
# 6 / 12    ---> accuracy: 0.5
# 7 / 12    ---> accuracy: 0.58

7 / 12
accuracy: 0.5833333333333334
